In [ ]:
#load padre executable into the environment
from hublib import use
from hublib.cmd import runCommand
from simtool import DB
import math
import re
import uuid
import os, glob
import pathlib
import simtool

%use padre-2.4E-r15
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

p_node:
    type : Integer
    description : Number of plotted points along the length of the P-type material. Increase number of nodes at high doping for better accuracy 
    value : 60
    min : 0
    max : 1000
        
n_node:
    type : Integer
    description : Number of plotted points along the length of the N-type material. Increase number of nodes at high doping for better accuracy.
    value : 120 
    min : 0
    max : 1000
        
i_node:
    type : Integer
    description : Number of plotted points along the length of the intrinsic (zero doping) region. Only valid when intrinsic region length is not zero.
    value : 0
    min : 0 
    max : 1000
        
p_len:
    type : Number
    current : 3.0, 
    description : Physical length of P-type material.
    units : um
    value : 3 
    min : 0.0
    max : 100.0
                            
n_len:
    type : Number
    description : Physical length of N-type material. 
    units : um
    value : 6
    min : 0.0
    max : 100.0
        
i_len:
    type : Number
    value : Physical length of the intrinsic region (zero doping).
    units : um
    value : 0
    min : 0.0
    max : 100.0
        
temperature:
    type : Number
    value : 300.0, 
    description : This is the temperature in the environment around the device.
    units : K
    value : 300
    min : 77.0
    max : 500.0
        
materialp:
    type : Text
    value : Si
    description : Set the type of material used for the entire region.
    value : Si
    options : [GaAs, Ge, Si, InP]
                        
vsweep_high:
    type : Number
    value : 0.6 
    description : Set an applied voltage across the PIN Junction.  For IV characteristics, the voltage will sweep from 0V (ground) to the applied voltage.
    units : V 
    value : 0.6 
    min : -5.0
    max : 5.0
        
vn_step:
    type : Integer
    value : 20
    description : Set the number data points sampled in the voltage sweep.
    label : Number of points
    min : 2
        
taun:
    type : Number
    description : This is the minority carrier lifetime for electrons.  It represents the average time that an electron will diffuse in p-type material before recombining with a hole.
    label : For electrons
    units : second
    value : 1.e-10
    min : 0.0

taup:
    type : Number
    description : This is the minority carrier lifetime for holes.  It represents the average time that a hole will diffuse in n-type material before recombining with an electron.
    label : For holes 
    units : second
    value : 1.e-10 
    min : 0.0
        
impurity:
    type : Boolean
    description : Here you define if any impurity doping level exists in the intrinsic region.
    label : Impurity doping in Intrinsic region. 
    value : False
                
impuritydoping:
    type : Text
    description : Here you define type of doping for in the intrinsic region
    label : Type of doping 
    value : Ptype 
    options : [Ntype, Ptype]
        
impuritylevel:
    type : Number
    description : Unintentional doping in the intrinsic region.
    label : Doping level 
    units : cm^-3
    value : 1.e+13
    min : 0.0
    max : 1.e+26

Na:
    type : Number
    value : 2.e+15
    description : Acceptor concentration (Na-)
    units : cm^-3
    min : 1.0
    max : 2.e+22

Nd:
    type : Number
    value : 1.e+16
    description : Donor concentration (Nd+)
    units : cm^-3 
    min : 1.0
    max : 2.e+22

In [ ]:
def FUNCTION_TO_INCLUDE(inputs):
    params = simtool.utils.parse(inputs)
    for param in params:
        globals()[param]=params[param].value
simtool.utils.validateInputs = FUNCTION_TO_INCLUDE     
simtool.utils.validateInputs(INPUTS)

In [ ]:
np =  p_node
nn = n_node
ni = i_node
lp = p_len
ln = n_len
li = i_len
tempk = temperature
mname = materialp
vhigh = vsweep_high
vstep_nm = vn_step
taun = taun
taup = taup
impure = impurity
impuretype = impuritydoping
impuredop = impuritylevel
ndargs = Nd
naargs = Na

In [ ]:
vlow = 0.000
vstep_sz = (vhigh - vlow)/(vstep_nm-1)
vcurr = 0
#TODO this should throw an error
if ln == 0:
    nn = 0
if lp == 0:
    np = 0
if li == 0:
    ni = 0
if ni == 0:
    li = 0

# constants
kbT = 1.32e-23*tempk
q = 1.602e-19
EPSILON = 11.8
EPSILON_0 = 8.85e-12

#
N_max = naargs*1e6
if abs(ndargs) > abs(naargs):
    N_max = ndargs*1e6
    
N_diff  = ndargs - naargs

large_ndiff_flag = 0
if N_diff > N_max*1/5:
    large_ndiff_flag = 1

p_eg= 0.0
names = {"InP":"inp", "Si": "silicon", "GaAs":"gaas", "Ge":"germanium"}
eg = {"InP":1.35, "Si": 1.12, "GaAs":1.4, "Ge":0.7}
mname = names[materialp]
p_eg = eg[materialp]
np_left = math.ceil(np*0.1)
nn_left = math.floor(nn/10*9)+ni+np
pdrfile = str(uuid.uuid4().hex)


In [ ]:
# determine which method is best for mesh generation
debye_l = math.sqrt((EPSILON*EPSILON_0*kbT)/(q*q*N_max))*1e6 
x_o = debye_l/5

# refining parameter calculation function
def r_cal (r_in, xot, n, xt):
    flag_conv=0
    if (n > 1):
        r = r_in
        while flag_conv == 0:
            if (r == 1):
                rnum = n*xot-xt
                denom = 0.5*xot*n*(n-1)
            else:
                term1 = 1/(r-1)
                term2 = pow(r,n)-1
                rnum = xot*term1*term2-xt
                term3 = n*pow(r,(n-1))
                denom = xot*(term1*term3-term1*term1*term2)

            r_new = r-rnum/denom
            error = abs(r-r_new)/abs(r_new)
            if error < 1e-15:
                flag_conv = 1
            else:
                r = r_new

            if r < 0:
                r = 1
    else:
        r = 1
    return r

r_n = r_cal(2, x_o, nn/2, ln/2)
r_p = r_cal(2, x_o, np/2, lp/2)


In [ ]:
deck =  "title      pn diode (setup)\n"
deck +=  "options    PO\n"
deck += "\n"
deck += "\n"
deck += "mesh       rect nx={nx} ny=3 width=1 outf={pdr}.mesh\n".format( nx = np+ni+nn, pdr=pdrfile )
deck += "x.m        n={n} l={l} r={r}\n".format( n=1, l=0, r=1 )
if lp != 0:
    if lp < 400:
        deck +=  "x.m        n={n} l={l} r={r}\n".format( n=np_left, l=lp/2.0, r=0.95 )
        if large_ndiff_flag == 1 :
            deck += "x.m        n={n} l={l} r={r}\n".format( n=np, l=lp, r=r_n )
        else :
            deck += "x.m        n={n} l={l} r={r}\n".format( n=np, l=lp, r=0.95 )
    else:
        deck +=  "x.m        n={n} l={l} r={r}\n".format( n=np, l=lp, r=0.95)

if li != 0:
    deck +=  "x.m        n={n} l={l} r={r}\n".format( n=np+int(ni/2), l=lp+li/2.0, r=1.11)
    deck +=  "x.m        n={n} l={l} r={r}\n".format( n=np+ni, l=lp+li, r=0.9)

if ln != 400:
    if ln < 400:
        if large_ndiff_flag == 1:
            deck += "x.m        n={n} l={l} r={r}\n".format( n=nn_left, l=lp+li+ln/2.0, r=1-r_p )
        else:
            deck += "x.m        n={n} l={l} r={r}\n".format( n=nn_left, l=lp+li+ln/2.0, r=1.11 )
        deck += "x.m        n={n} l={l} r={r}\n".format( n=np+ni+nn, l=lp+li+ln, r=1.05 )
    else:
        deck += "x.m        n={n} l={l} r={r}\n".format( n=np+ni+nn, l=lp+li+ln, r=1.05 )

deck += "y.m        n={n} l={l} r={r}\n".format( n=1, l=0, r=1 )
deck += "y.m        n={n} l={l} r={r}\n".format( n=3, l=1, r=1 )
deck += "\n"

'''foreach box [$opt children -type box input.structure.current.components] { TODO ANALYZE
    if {$box == "box"} {
        set box box0
    }
    set z0 [$opt get input.structure.current.components.$box.corner]
    set z0 [Rappture::Units::convert $z0 -to um -units off]

    set z1 [$opt get input.structure.current.components.$box.corner1]
    set z1 [Rappture::Units::convert $z1 -to um -units off]

    set tm [expr {$z1-$z0}]
}'''

deck += "$ Regions specifications\n"

if lp != 0:
    deck += "region     num={num} ix.l={ixl} ix.h={ixh} iy.l={iyl} iy.h={iyh} MATER={mater} SEMI\n".format( num=1, ixl=1, ixh=np, iyl=1, iyh=3, mater=mname )
if li != 0:
    deck += "region     num={num} ix.l={ixl} ix.h={ixh} iy.l={iyl} iy.h={iyh} MATER={mater} SEMI\n".format( num=1, ixl=np, ixh=np+ni, iyl=1, iyh=3, mater=mname )
if ln != 0:
    deck += "region     num={num} ix.l={ixl} ix.h={ixh} iy.l={iyl} iy.h={iyh} MATER={mater}\n".format( num=1, ixl=np+ni, ixh=np+ni+nn, iyl=1, iyh=3, mater=mname )



deck +=  "elec       num=1 ix.l=1 ix.h=1 iy.l=1 iy.h=3\n"
deck +=  "elec       num=2 ix.l={ixl} ix.h={ixh} iy.l=1 iy.h=3\n".format( ixl = np+ni+nn, ixh = np+ni+nn)
deck +=  "\n"

deck +=  "$ Doping specification\n"
if lp != 0:
    deck +=  "dop        reg=1 p.type conc={conc} x.l={xl} x.r={xr} y.top=0 y.bot=1 uniform\n".format(conc=naargs, xl=0, xr=lp ) 

if impure and li != 0 and impuretype == "Ntype":
    deck +=  "dop        reg=1 n.type conc={conc} x.l={xl} x.r={xr} y.top=0 y.bot=1 uniform\n".format(conc=impuredop, xl=lp, xr=lp+li )  
elif impure and li != 0 and impuretype == "Ptype":
    deck +=  "dop        reg=1 p.type conc={conc} x.l={xl} x.r={xr} y.top=0 y.bot=1 uniform\n".format(conc=impuredop, xl=lp, xr=lp+li )

if ln != 0:
    deck += "dop        reg=1 n.type conc={conc} x.l={xl} x.r={xr}\n".format(conc=ndargs, xl=lp+li, xr=lp+li+ln )
    deck += "+y.top=0 y.bot=1 uniform\n"
deck += "plot.1d    log dop abs a.x=0 b.x={bx} b.y=0.5 a.y=0.5 points ascii\n".format(bx=lp+li+ln)
deck += "+          outf={pdr}.dop\n".format(pdr = pdrfile)
deck += "\n"

deck += "$ MATERIAL SPECIFICATIONS\n"
deck += "material name={name} taun0={taun0} taup0={taup0} trap.typ=0 etrap=0\n".format(name=mname , taun0=taun, taup0=taup)

deck += "$ Ref: ASF Vol 6 R.F.Pierret\n"
deck += "\n"

if mname == "silicon":
    deck += "+ EG300=1.11676 PERMITTIVITY=11.8 NC300=3.2316E19 NV300=1.812063E19\n"
elif mname == "gaas":
    deck += "+ EG300=1.42 PERMITTIVITY=13.1 NC300=4.21E17 NV300=9.52E18\n"
elif mname == "germanium":
    deck += "+ EG300=0.66 PERMITTIVITY=16.0 NC300=1.03E19 NV300=5.35E18\n"
elif mname == "inp":
    deck += " \n"
    deck += "MATERIAL NAME=inp DEF=gaas \n"
    deck += "+ EG300=1.350 \n"
    deck += "+ EGAlpha=3.63e-4 \n"
    deck += "+ EGBeta=162 \n"
    deck += "+ Permittivity=12.56 \n"
    deck += "+ NC300=5.7e17 \n"
    deck += "+ NV300=1.1e19 \n"
    deck += "+ MUN0=5300 \n"
    deck += "+ MUP0=200 \n"
    deck += "+ VSATN=6.7e6 \n"
    deck += "+ VSATP=5e6 \n"

deck += "\n"

deck += "$ Specify models\n"
deck += "contact all neutral\n"
deck += "models srh conmob fldmob impact temperature={temperature}\n".format(temperature=tempk)
deck += "system electrons holes newton\n"
deck += "\n"
deck += "$ Solve for initial conditions\n"
deck += "solve init\n"
deck += "plot.1d pot a.x=0 b.x={bx} \n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.pot\n".format(pdr = pdrfile)
deck += "plot.1d band.val a.x=0 b.x={bx} \n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.vband\n".format(pdr = pdrfile)
deck += "plot.1d band.con a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.cband\n".format(pdr = pdrfile)
deck += "plot.1d qfn a.x=0 b.x={bx}\n ".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.qfn\n".format(pdr = pdrfile)
deck += "plot.1d qfp a.x=0 b.x={bx}\n ".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.qfp\n".format(pdr = pdrfile)
deck += "plot.1d ele a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.ele\n".format(pdr = pdrfile)
deck += "plot.1d hole a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.hole\n".format(pdr = pdrfile)
deck += "plot.1d net.charge a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.ro\n".format(pdr = pdrfile)
deck += "plot.1d e.field a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}.efield\n".format(pdr = pdrfile)
deck += "\n"
deck += "$ Solve for applied bias\n"
deck += "log outf={pdr}.iv\n".format(pdr = pdrfile)
deck += "solve prev\n"


counter = 1
while counter <= vstep_nm :
    deck += "$ Solve for applied bias\n"
    deck += "solve v1={v1}  elect=1\n".format(v1=vstep_sz*(counter-1))
    deck += "plot.1d pot a.x=0 b.x={bx} \n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.potiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d band.val a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.vbiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d band.con a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.cbiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d qfn a.x=0 b.x={bx} \n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.qfniv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d qfp a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.qfpiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d ele a.x=0 b.x={bx} \n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.eleiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d hole a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.holeiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d net.charge a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.roiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d e.field a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.efieldiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d recomb a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.reiv\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d j.electr a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.jelectr\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d j.hole a.x=0 b.x={bx} \n".format(bx=lp+li+ln)
    deck += "+a.y=0.5 b.y=0.5 ascii outf={pdr}_bias_{counter}.jhole\n".format(pdr = pdrfile, counter=counter)
    deck += "plot.1d j.total a.x=0 b.x={bx}\n".format(bx=lp+li+ln)
    deck += "+ a.y=0.5 b.y=0.5  ascii  outf={pdr}_bias_{counter}.jtot\n".format(pdr = pdrfile, counter=counter)
    deck += "\n"
    counter = counter + 1

deck += "log off\n"
deck += "log acfile={pdr}.ac\n".format(pdr = pdrfile)
deck += "solve v1=0 vstep={vstep} nstep={nstep} elect=1\n".format(vstep=vstep_sz, nstep=vstep_nm)
deck += "+     ac freq=10\n"
deck += "log off\n\n"
deck += "end\n"

#print(deck)

In [ ]:
deckfile = pdrfile
with open(deckfile, "w") as fid:
    fid.write(deck)
status,out,error = runCommand("padre " + deckfile)

In [ ]:
if "PADRE Aborted" in out.decode():
    status = 1

In [ ]:
%%yaml OUTPUTS
Output Log:
    type: Text    
    description : Output Log

Parameters:
    type: Text    
    description : Parameters
    
Efn:
    type: Dict
    description : quasi Fermi level for electrons at equilibrium
    
Efp:
    type: Dict
    description : quasi Fermi level for holes at equilibrium

Hole Density:
    type: Dict
    description : Hole Density at equilibrium
    
Electron Density:
    type: Dict
    description : Electron Density at equilibrium
    
Equilibrium Hole Density:
    type: Dict
    description : Hole Density
    
Equilibrium Electron Density:
    type: Dict
    description : Electron Density 

Charge Density:
    type: Dict
    description: Charge Density (at equilibrium)

Equilibrium Net Charge Density:
    type: Dict
    description: Net Charge Density (at equilibrium)
    
Equilibrium Ei:
    type: Dict
    description: Ei
    
Equilibrium Potential:
    type: Dict
    description: Electrostatic Potential (at equilibrium)
    
Equilibrium Ec :
    type: Dict
    description: Ec (at equilibrium)

Ec :
    type: Dict
    description: Ec

Equilibrium Ev:
    type: Dict
    description: Ev (at equilibrium)
    
Ev:
    type: Dict
    description: Ev
    
Equilibrium Electric Field:
    type: Dict
    description: Electric Field (at equilibrium)
    
Electric Field:
    type: Dict
    description: Electric Field
    
Equilibrium Recombination Rate:
        type: Dict
        description: Recombination Rate (at equilibrium)

Recombination Rate:
    type: Dict
    description: Recombination Rate

CV Characteristic:
    type: Dict
    description: voltage -> Gate voltage (V), function -> Capacitance (F/cm2)

IV Characteristic:
    type: Dict
    description: voltage -> Gate voltage (V), function -> Current (A/cm2)

QuasiFermi Electrons:
    type: Dict
    description: Quasi Fermi level for electrons

QuasiFermi Holes:
    type: Dict
    description: Quasi Fermi level for holes
    
Total Current Holes:
    type: Dict
    description: Total current holes

Total Current Electrons:
    type: Dict
    description: Total current electrons

Total Current:
    type: Dict
    description: Total current

Excess Electron Density:
    type: Dict
    description: Excess Electron Density
    
Excess Hole Density:
    type: Dict
    description: Excess Hole Density
    
Net Charge Density:
    type: Dict
    description: Net Charge Density
    
Electrostatic Potential:
    type: Dict
    description: Electrostatic Potential

Ei:
    type: Dict
    description: Energy Band Diagram

In [ ]:
def getData(filename):
    data = {"position":[], "function":[]}
    try:
        with open(filename, "r") as info:
            lines = re.sub('([0-9])([-+][0-9])','\g<1>E\g<2>', info.read())
            for i, line in enumerate(lines.splitlines()):
                regex = (re.findall(r'-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+]?\ *[0-9]+)?', line))
                if len(regex)==4:
                    data['position'].append(float(regex[0].strip()))
                    data['function'].append(float(regex[1].strip()))
    except:
        data = {"position":[], "function":[]}
    return data

def getSequence(filename, samples, step, precision = 6):
    sequence = {}
    for counter in range(samples):
        data = {"position":[], "function":[]}
        sample = "{:.{}f}".format( (counter)*step, precision )
        data = getData(filename.format(sample = counter+1))
        sequence[sample]=data
    return sequence

db = DB(OUTPUTS)


In [ ]:
params = "== STRUCTURE ==\n"
params += "  P-region:  {lp} um ({np} nodes) at {naargs}/cm3\n".format(lp=lp, np=np, naargs=naargs)
if impure:
    params += "  I-region:  {li} um ({ni} nodes) at {impuredop}/cm3\n".format(li=li, ni=ni, impuredop=impuredop)
else:
    params += "  I-region:  {li}um ({ni} nodes)\n".format(li=li, ni=ni)

params += "  N-region:  {ln} um ({nn} nodes) at {ndargs}/cm3\n".format(ln=ln, nn=nn, ndargs=ndargs)
params += "\n== MATERIALS ==\n".format()
params += "  Material: {mname}\n".format(mname=mname)
params += "  taun = {taun}s\n".format(taun=taun)
params += "  taup = {taup}s\n".format(taup=taup)
params += "\n== ENVIRONMENT ==\n"
params += "  Temperature: {tempk}K\n".format(tempk=tempk)
params += "  V bias: {vhigh} V ({vstep_nm} points)\n".format(vhigh=vhigh,vstep_nm=vstep_nm)
db.save('Parameters', params)

In [ ]:
db.save('Output Log', out.decode())

In [ ]:
qfndata = getData(deckfile+".qfn")
db.save('Efn', qfndata)

In [ ]:
db.save('Efp', getData(deckfile+".qfp"))

In [ ]:
hole = getData(deckfile+".hole")
db.save('Equilibrium Hole Density', hole)

In [ ]:
ele = getData(deckfile+".ele")
db.save('Equilibrium Electron Density', ele)

In [ ]:
db.save('Charge Density', getData(deckfile+".dop"))

In [ ]:
db.save('Equilibrium Net Charge Density', getData(deckfile+".ro"))

In [ ]:
pot = getData(deckfile+".pot")
pot['function'] = [-1*f for f in pot['function']]
db.save('Equilibrium Ei', pot)

In [ ]:
def getDataDelta(data):
    data_delta = {"position" : [], "delta" : []}
    for i,v in enumerate (data['position']):
        if (i>0):
            data_delta['delta'].append(-data['function'][i]+data['function'][0]) #potential is negative after change in previous cell
        else:
            data_delta['delta'].append(0)
        data_delta['position'].append(data['position'][i])
    return data_delta

In [ ]:
db.save('Equilibrium Potential', getDataDelta(pot))

In [ ]:
ec = getData(deckfile+".cband")
ec['function'] = [-1*f for f in ec['function']]
db.save('Equilibrium Ec', ec)

In [ ]:
ev = getData(deckfile+".vband")
ev['function'] = [-1*f for f in ev['function']]
db.save('Equilibrium Ev', ev)

In [ ]:
efield = getData(deckfile+".efield")
efield['function'] = [-1*f for f in efield['function']]
db.save('Equilibrium Electric Field', efield)

In [ ]:
recomb = getData(deckfile+".recomb")
recomb['function'] = [-1*f for f in recomb['function']]
db.save('Equilibrium Recombination Rate', recomb)

In [ ]:
def getDataAC(filename):
    data = {"voltage":[], "function":[]}
    try:
        with open(filename, "r") as info:
            lines = re.sub('([0-9])([-+][0-9])','\g<1>E\g<2>', info.read())
            first = 0
            vg_data = 0
            for i, line in enumerate(lines.splitlines()):
                regex = (re.findall(r'-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+]?\ *[0-9]+)?', line))  
                if len(regex)==4:
                    vg_data = float(regex[0].strip())
                if len(regex)==5:
                    if float(regex[0].strip()) == 1:
                        data['voltage'].append(vg_data)
                        data['function'].append(abs(float(regex[3].strip()))*1e8)                            
    
    except:
        data = {"position":[], "function":[]}
    return data
ac = getDataAC(deckfile+".ac")
db.save('CV Characteristic', ac)

In [ ]:
def getDataIV(filename):
    data = {"voltage":[], "function":[]}
    insert = True
    try:
        with open(filename, "r") as info:
            lines = re.sub('([0-9])([-+][0-9])','\g<1>E\g<2>', info.read())
            first = 0
            vg_data = 0
            for i, line in enumerate(lines.splitlines()):
                regex = (re.findall(r'-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+]?\ *[0-9]+)?', line))  
                if len(regex)==5:
                    if insert:
                        data['voltage'].append(float(regex[0].strip()))
                        data['function'].append(float(regex[4].strip())*1e8)
                    insert = not insert
    
    except:
        data = {"position":[], "function":[]}
    return data
iv = getDataIV(deckfile+".iv")
db.save('IV Characteristic', ac)

In [ ]:
qfniv = getSequence(pdrfile+"_bias_{sample}.qfniv", vstep_nm, vstep_sz)
qfe = {}
for k in qfniv.keys():
    v = qfniv[k]
    v['function'] = [-1*f for f in qfniv[k]['function']]
    qfe[k] = v
db.save('QuasiFermi Electrons', qfe)

In [ ]:
qfpiv = getSequence(pdrfile+"_bias_{sample}.qfpiv", vstep_nm, vstep_sz)
qfh = {}
for k in qfpiv.keys():
    v = qfpiv[k]
    v['function'] = [-1*f for f in qfpiv[k]['function']]
    qfh[k] = v
db.save('QuasiFermi Holes', qfh)


In [ ]:
hcurr = getSequence(pdrfile+"_bias_{sample}.jhole", vstep_nm, vstep_sz)
db.save('Total Current Holes', hcurr)


In [ ]:
ecurr = getSequence(pdrfile+"_bias_{sample}.jelectr", vstep_nm, vstep_sz)
db.save('Total Current Electrons', ecurr)


In [ ]:
totcurr = getSequence(pdrfile+"_bias_{sample}.jtot", vstep_nm, vstep_sz)
db.save('Total Current', totcurr)

In [ ]:
ndata = getSequence(pdrfile+"_bias_{sample}.eleiv", vstep_nm, vstep_sz)
db.save('Electron Density', ndata)

In [ ]:
pdata = getSequence(pdrfile+"_bias_{sample}.holeiv", vstep_nm, vstep_sz)
db.save('Hole Density', pdata)

In [ ]:
def getDataDiff(data, data0):
    exndata = {}
    for nd in data:
        a0last = 0
        n0last = 0
        datav = {"position":[], "function":[]}
        for k,v in enumerate(data0["position"]):
            a0 = data0["position"][k]
            n0 = data0["function"][k]    
            a = data[nd]["position"][k]
            n = data[nd]["function"][k]
            datav["position"].append(a)
            if a0 == a or k==0:
                datav["function"].append(n-n0)
            else: #interpolate if the position is not exactly
                a0last = data0["position"][k-1]
                n0last = data0["function"][k-1]
                slope = (n0-n0last)/(a0-a0last)
                n0interp = slope * (a-a0last) + n0last
                datav["function"].append(n-n0interp)
        exndata[nd] = datav
    return exndata

In [ ]:
exn = getDataDiff(ndata, ele)
db.save('Excess Electron Density', exn)

In [ ]:
exp = getDataDiff(pdata, hole)
db.save('Excess Hole Density', exp)


In [ ]:
rov = getSequence(pdrfile+"_bias_{sample}.roiv", vstep_nm, vstep_sz)
db.save('Net Charge Density', rov)

In [ ]:
potiv = getSequence(pdrfile+"_bias_{sample}.potiv", vstep_nm, vstep_sz)
ei = {}
for k in potiv.keys():
    v = potiv[k]
    v['function'] = [-1*f for f in potiv[k]['function']]
    ei[k] = v
    potiv[k] = getDataDelta(v)
db.save('Ei', ei)
db.save('Electrostatic Potential', potiv)

In [ ]:
cbiv = getSequence(pdrfile+"_bias_{sample}.cbiv", vstep_nm, vstep_sz)
ec = {}
for k in cbiv.keys():
    v = cbiv[k]
    v['function'] = [-1*f for f in cbiv[k]['function']]
    ec[k] = v
db.save('Ec', ec)

In [ ]:
vbiv = getSequence(pdrfile+"_bias_{sample}.vbiv", vstep_nm, vstep_sz)
ev = {}
for k in vbiv.keys():
    v = vbiv[k]
    v['function'] = [-1*f for f in vbiv[k]['function']]
    ev[k] = v
db.save('Ev', ev)

In [ ]:
efieldiv = getSequence(pdrfile+"_bias_{sample}.efieldiv", vstep_nm, vstep_sz)
efv = {}
for k in efieldiv.keys():
    v = efieldiv[k]
    v['function'] = [-1*f for f in efieldiv[k]['function']]
    efv[k] = v
db.save('Electric Field', efv)

In [ ]:
reiv = getSequence(pdrfile+"_bias_{sample}.reiv", vstep_nm, vstep_sz)
rr = {}
for k in reiv.keys():
    v = reiv[k]
    v['function'] = [-1*f for f in reiv[k]['function']]
    rr[k] = v
db.save('Recombination Rate', rr)

In [ ]:
for filename in glob.glob(pdrfile + "*"):
    os.remove(filename) 